## Create models for real time inference



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive
%mkdir infer_mt
%cd infer_mt

In [ ]:
!pip install -q ctranslate2 OpenNMT-py sentencepiece
!pip install -q gradio transformers torch

In [3]:
build_directory=!pwd
input_model="../transformer/models/model_step_10000.pt"
mtmodel="enga"

In [4]:
import gradio as gr
import ctranslate2
import sentencepiece as spm

In [5]:
# Convert OpenNMT model using CTranslate2 and copy SentencePiece model
!ct2-opennmt-py-converter --model_path $input_model --output_dir $mtmodel
!cp ../transformer/data/spm.model ./enga

## Launch Gradio and run translation interface

In [6]:
LANGS = ["eng_Latn", "gle_Latn"]
MODEL_LIST = ["enga"]

def translate(text, src_lang, tgt_lang, cmodel):
    """ Translate the text from source lang to target lang """
    translator = ctranslate2.Translator(str(cmodel),device="cpu")

    sp = spm.SentencePieceProcessor(str(cmodel)+"/spm.model")
    input_tokens = sp.encode(text, out_type=str)
    results = translator.translate_batch([input_tokens])
    output_tokens = results[0].hypotheses[0]
    result = sp.decode(output_tokens)

    return result

In [7]:
demo = gr.Interface(
    fn=translate,

     inputs=[
        gr.components.Textbox(label="Text"),
        gr.components.Dropdown(label="Source Language", choices=LANGS),
        gr.components.Dropdown(label="Target Language", choices=LANGS),
        gr.components.Dropdown(label="Choose Model", choices=MODEL_LIST),
    ],

    outputs=["text"],
    examples=[["I am sick today", "eng_Latn", "gle_Latn"]],
    cache_examples=False,
    title="Translation Automation Demo",
    description="To record any interesting outputs in flagged/log.csv, just hit the Flag button"
)

In [8]:
demo.launch(debug=True)